# LFT 01 — Introduction (Rigor Edition)

This notebook introduces **Logic Field Theory (LFT)** with **fully justified derivations** and verifications on the minimal nontrivial case **N=3**. We formalize the logical filtering pipeline (A = L(I)), connect it to **Coxeter system** $W(A_{N-1})$, and verify claims with exact code.

### Roadmap
1. **Logical Filtering** (Identity, Non-Contradiction, Excluded Middle) → DAGs/partials/totals.
2. **Coxeter Derivation**: $S_N \cong W(A_{N-1})$ acts on the sum-zero space $V$ (dim = $N-1$).
3. **Permutohedron**: vertices = total orders, **edges = adjacent transpositions**.
4. **Verification for $N=3$**: pattern counts, $A_2$ Gram matrix, Cayley graph properties, and the permutohedron hexagon.


## 1. Logical Filtering: from raw distinctions to orders

**Definition.** A *pattern* on elements $\{0,\dots,N-1\}$ is a set of ordered pairs $E\subseteq \{0,\dots,N-1\}^2$. The logical operator $L$ enforces:
- **Identity (ID)**: $(i,i)$ for all $i$ (reflexivity).
- **Non-Contradiction (NC)**: never both $(i,j)$ and $(j,i)$ for $i\ne j$.
- **Excluded Middle (EM)** *(operational proxy)*: **acyclicity** (so the non-reflexive edges form a DAG).

Thus a pattern passes $L$ iff its non-reflexive edges define a **partial order**. **Total orders** are DAGs where for each unordered pair $\{i,j\}$ exactly one orientation appears.

In [ ]:
import numpy as np
import networkx as nx
from itertools import combinations

def has_cycle(pattern):
    edges = [(a,b) for (a,b) in pattern if a!=b]
    G = nx.DiGraph()
    G.add_edges_from(edges)
    return not nx.is_directed_acyclic_graph(G)

def violates_non_contradiction(pattern):
    pairs = set((a,b) for (a,b) in pattern if a!=b)
    for (a,b) in pairs:
        if (b,a) in pairs:
            return True
    return False

def apply_L(pattern, elements):
    # Identity
    p = set(pattern) | {(i,i) for i in elements}
    # Non-Contradiction
    if violates_non_contradiction(p):
        return None
    # Excluded Middle (acyclicity)
    if has_cycle(p):
        return None
    return p

def is_total_order(pattern, elements):
    p = set((a,b) for (a,b) in pattern if a!=b)
    # exactly one orientation per unordered pair
    for i,j in combinations(elements,2):
        if ((i,j) in p) == ((j,i) in p):
            return False
    # acyclic
    return nx.is_directed_acyclic_graph(nx.DiGraph(list(p)))

def pattern_to_ordering(pattern, elements):
    if not is_total_order(pattern, elements):
        return None
    G = nx.DiGraph([(a,b) for (a,b) in pattern if a!=b])
    order = list(nx.topological_sort(G))
    return '<'.join(map(str, order))

### Exhaustive classification for $N=3$
We enumerate **all directed-pair subsets** (with reflexives enforced by ID) and classify into cycles, partials, and totals.

In [ ]:
from itertools import product
import matplotlib.pyplot as plt
import os

N=3
elements=tuple(range(N))
dirs=[(a,b) for a in elements for b in elements if a!=b]  # 6 directed pairs

all_patterns=[]
for mask in range(1<<len(dirs)):
    subset=[dirs[k] for k in range(len(dirs)) if (mask>>k)&1]
    pattern=set(subset) | {(i,i) for i in elements}
    all_patterns.append(pattern)

filtered=[p for p in all_patterns if apply_L(p, elements) is not None]
totals=[p for p in filtered if is_total_order(p, elements)]
partials=[p for p in filtered if not is_total_order(p, elements)]
cycles=[p for p in all_patterns if has_cycle(p)]

print({'total':len(all_patterns),'consistent':len(filtered),'partials':len(partials),'totals':len(totals),'cycles':len(cycles)})

os.makedirs('./outputs', exist_ok=True)
labels=['Cycles','Partials','Totals']
sizes=[len(cycles),len(partials),len(totals)]
plt.figure()
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
plt.title('Pattern Distribution (N=3)')
plt.tight_layout()
plt.savefig('./outputs/N3_pattern_distribution.png', dpi=150)
plt.close()

## 2. Coxeter derivation: $S_N \cong W(A_{N-1})$ on $V=\{x: \sum x_i=0\}$

**Theorem (standard).** With simple roots $\alpha_i=e_i-e_{i+1}$, the reflections
$$s_i(x)=x-\frac{2\langle x,\alpha_i\rangle}{\langle\alpha_i,\alpha_i\rangle}\,\alpha_i$$
swap coordinates $i\leftrightarrow i+1$. The generated group is isomorphic to $S_N$, and $\dim V=N-1$ gives the **rank**.

**Check for $A_2$ (N=3):** the Gram/Cartan matrix has diagonals 2, off-diagonals −1 for adjacent roots (cosine −1/2 = 120°).

In [ ]:
import numpy as np

def roots_A(N):
    # simple roots alpha_i = e_i - e_{i+1} in R^N
    roots=[np.eye(N)[i]-np.eye(N)[i+1] for i in range(N-1)]
    roots=np.stack(roots, axis=0)  # (N-1, N)
    G=np.zeros((N-1,N-1))
    for i in range(N-1):
        for j in range(N-1):
            G[i,j]=2*np.dot(roots[i],roots[j])  # Cartan form: 2*(,)/(||alpha||^2) with ||alpha||^2=2
    return roots,G

roots,G=roots_A(3)
print('Cartan A2:\n',G)
cos=G/2.0
print('Cosines:\n',cos)

## 3. Permutohedron & Cayley graph (edges = adjacent swaps)

**Definition.** For strictly increasing $a\in\mathbb{R}^N$ with $\sum a_i=0$, the **permutohedron** $\Pi_{N-1}(a)=\operatorname{conv}\{\sigma\cdot a\}$ lives in $V$. Its 1-skeleton connects permutations that differ by a **single adjacent transposition** (Coxeter generator).

We verify this for $N=3$ by constructing the **adjacent-generator** Cayley graph on $S_3$ and checking degree/edge counts and connectivity.

In [ ]:
import itertools, networkx as nx

def cayley_adjacent_graph(N):
    perms=list(itertools.permutations(range(N)))
    idx={p:i for i,p in enumerate(perms)}
    G=nx.Graph()
    G.add_nodes_from(range(len(perms)))
    gens=[(i,i+1) for i in range(N-1)]
    for p in perms:
        u=idx[p]
        for (i,j) in gens:
            q=list(p); q[i],q[j]=q[j],q[i]
            v=idx[tuple(q)]
            if u<v:
                G.add_edge(u,v)
    return G, perms

G3, perms3 = cayley_adjacent_graph(3)
print({'nodes':G3.number_of_nodes(),'edges':G3.number_of_edges(),'avg_degree':2*G3.number_of_edges()/G3.number_of_nodes()})
assert G3.number_of_nodes()==6
assert G3.number_of_edges()==6  # N!*(N-1)/2 for N=3 → 6*2/2=6
assert nx.is_connected(G3)
print('Cayley S3 (adjacent) checks passed.')

### Visual: permutohedron $\Pi_2$ (N=3) as a hexagon (PCA on $V$)
We embed the vertex set (permutations of a centered vector) in the sum-zero space and plot a 2D PCA projection to show the hexagon structure.

In [ ]:
import numpy as np, matplotlib.pyplot as plt

def sum_zero_basis(N):
    diffs=np.zeros((N,N-1))
    for i in range(N-1):
        diffs[i,i]=1.0; diffs[i+1,i]=-1.0
    U,S,Vt=np.linalg.svd(diffs, full_matrices=False)
    return U  # N x (N-1)

def permutohedron_coords(N):
    B=sum_zero_basis(N)
    a=np.arange(N, dtype=float)-(N-1)/2.0
    verts=[]
    for p in itertools.permutations(range(N)):
        verts.append(B.T @ a[list(p)])
    V=np.vstack(verts)  # (N!, N-1)
    X=V - V.mean(axis=0, keepdims=True)
    U,S,Vt=np.linalg.svd(X, full_matrices=False)
    P2=X @ Vt[:2].T
    return V,P2

V,P2=permutohedron_coords(3)
plt.figure()
plt.scatter(P2[:,0],P2[:,1], s=40)
plt.title('Permutohedron $\\Pi_2$ (N=3) — PCA projection')
plt.axis('equal')
plt.tight_layout()
plt.savefig('./outputs/N3_permutohedron_hexagon.png', dpi=150)
plt.close()
print('Saved ./outputs/N3_permutohedron_hexagon.png')

## 4. Takeaways for the N−1 derivation
- The state space of total orders lives on the **permutohedron** in the **sum-zero space** $V\cong\mathbb{R}^{N-1}$.
- Its **edges are adjacent reflections** (Coxeter generators), so the **rank** (= number of independent generators) is **$N-1$**.
- For $N=3$, this gives a clean, finitely checkable case: $A_2$ roots, hexagon permutohedron, and a 2-regular adjacent Cayley graph.

In **Notebook 2 (Geometry Derivation)** we crank this to $N=4$ to show the 3D permutohedron $\Pi_3$ and the full $A_3$ Cartan geometry — the first physics-relevant case (3D space).